# Reservoir Characterization & Decline Analysis — Full Pipeline

This notebook is a runnable pipeline for:
1. Loading synthetic data
2. Merging core and well logs (nearest depth)
3. Feature engineering
4. Training ML models: porosity (regression), permeability (regression, log-target), facies (classification)
5. SHAP explanations for model interpretability (optional)
6. Decline Curve Analysis (Arps fits per well) and EUR estimation
7. Saving trained models and summary outputs

Run cells sequentially. Some optional steps (SHAP) will gracefully skip if package not installed.


In [ ]:
# 1. Imports & configuration
import warnings
warnings.filterwarnings('ignore')
import os
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
import joblib
import matplotlib.pyplot as plt
import math
DATA_DIR = Path('.')
OUT_DIR = Path('outputs')
OUT_DIR.mkdir(exist_ok=True)
print('Working directory:', Path.cwd())


In [ ]:
# 2. Load datasets
logs = pd.read_csv(DATA_DIR / 'well_logs.csv')
core = pd.read_csv(DATA_DIR / 'core_data.csv')
prod = pd.read_csv(DATA_DIR / 'production_timeseries.csv')
wells = pd.read_csv(DATA_DIR / 'wells_meta.csv')
print('logs', logs.shape)
print('core', core.shape)
print('prod', prod.shape)
logs.head()


## 3. Merge core data to nearest log depth
We will match each core measurement to the nearest log depth sample for the same well, and extract log curves as features.

In [ ]:
# Prepare a function to find nearest log sample per core depth
def merge_core_to_logs(logs_df, core_df, window=0.5):
    # logs_df: must contain Well_ID and Depth_m
    merged_rows = []
    for wid, group in core_df.groupby('Well_ID'):
        logs_w = logs_df[logs_df['Well_ID']==wid]
        if logs_w.empty:
            continue
        depths = logs_w['Depth_m'].values
        for _, row in group.iterrows():
            d = row['Depth_m']
            idx = (np.abs(depths - d)).argmin()
            log_row = logs_w.iloc[idx]
            merged = {**row.to_dict(), **{f'log_{k}': v for k,v in log_row.to_dict().items()}}
            merged_rows.append(merged)
    return pd.DataFrame(merged_rows)

merged = merge_core_to_logs(logs, core)
print('merged shape:', merged.shape)
merged.head()


## 4. Feature engineering
- Use logs (GR, NPHI, RHOB, DT, RES) and simple derived features
- Prepare targets: Porosity, Permeability (log-transform), Facies


In [ ]:
df = merged.copy()
# Extract features from the log_* columns
features = ['log_GR','log_NPHI','log_RHOB','log_DT','log_RES']
for f in features:
    if f not in df.columns:
        df[f] = np.nan

df['log_GR_over_RHOB'] = df['log_GR'] / (df['log_RHOB'] + 1e-6)
df['log_NPHI_times_GR'] = df['log_NPHI'] * df['log_GR']

target_por = 'Porosity'
target_perm = 'Permeability_mD'
target_facies = 'log_Facies' if 'log_Facies' in df.columns else 'Facies'
df = df.dropna(subset=features + [target_por, target_perm])
print('Final training rows:', df.shape[0])
df.head()


## 5. Train/Test split and scaling


In [ ]:
X = df[ ['log_GR','log_NPHI','log_RHOB','log_DT','log_RES','log_GR_over_RHOB','log_NPHI_times_GR'] ].values
y_por = df[target_por].values
y_perm = df[target_perm].values
y_facies = df['Facies'].values if 'Facies' in df.columns else None

X_train, X_test, y_por_train, y_por_test = train_test_split(X, y_por, test_size=0.2, random_state=42)
_, _, y_perm_train, y_perm_test = train_test_split(X, y_perm, test_size=0.2, random_state=42)
if y_facies is not None:
    _, _, y_f_train, y_f_test = train_test_split(X, y_facies, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s = scaler.transform(X_test)
joblib.dump(scaler, OUT_DIR / 'scaler.joblib')
print('Shapes:', X_train_s.shape, X_test_s.shape)


## 6. Train Random Forest for Porosity (regression)


In [ ]:
rf_por = RandomForestRegressor(n_estimators=150, max_depth=12, random_state=42, n_jobs=-1)
rf_por.fit(X_train_s, y_por_train)
y_por_pred = rf_por.predict(X_test_s)
print('Porosity RMSE:', mean_squared_error(y_por_test, y_por_pred, squared=False))
print('Porosity R2:', r2_score(y_por_test, y_por_pred))
joblib.dump(rf_por, OUT_DIR / 'rf_porosity.joblib')


## 7. Train Random Forest for Permeability (regression on log-target)
Permeability is skewed; take log(1+perm) as target.

In [ ]:
y_perm_log = np.log1p(y_perm)
y_perm_log_train = np.log1p(y_perm_train)
y_perm_log_test = np.log1p(y_perm_test)

rf_perm = RandomForestRegressor(n_estimators=200, max_depth=14, random_state=42, n_jobs=-1)
rf_perm.fit(X_train_s, y_perm_log_train)
y_perm_log_pred = rf_perm.predict(X_test_s)
y_perm_pred = np.expm1(y_perm_log_pred)
print('Permeability RMSE (mD):', mean_squared_error(y_perm_test, y_perm_pred, squared=False))
print('Permeability R2:', r2_score(y_perm_test, y_perm_pred))
joblib.dump(rf_perm, OUT_DIR / 'rf_permeability.joblib')


## 8. Train Random Forest Classifier for Facies (if available)


In [ ]:
if 'Facies' in df.columns:
    clf = RandomForestClassifier(n_estimators=200, max_depth=16, random_state=42, n_jobs=-1)
    clf.fit(X_train_s, y_f_train)
    y_f_pred = clf.predict(X_test_s)
    print(classification_report(y_f_test, y_f_pred))
    joblib.dump(clf, OUT_DIR / 'rf_facies.joblib')
else:
    print('Facies column not available in merged dataset.')


## 9. Feature importance and SHAP explanations (optional)
SHAP can be slow; this cell will attempt to run it and skip if SHAP is not installed.


In [ ]:
feature_names = ['GR','NPHI','RHOB','DT','RES','GR_over_RHOB','NPHI_times_GR']
try:
    import shap
    explainer = shap.Explainer(rf_por.predict, X_train_s)
    shap_vals = explainer(X_test_s[:200])
    # Plot summary (will render inline in Jupyter)
    shap.summary_plot(shap_vals, features=X_test_s[:200], feature_names=feature_names)
except Exception as e:
    print('SHAP not available or failed:', e)


## 10. Decline Curve Analysis (Arps) — functions and per-well fitting
We will fit simple Arps (exponential and hyperbolic) to each well's oil rate and select the best by RMSE.

In [ ]:
from scipy.optimize import minimize

def arps_rate(qi, d, b, t):
    t = np.asarray(t)
    if abs(b) < 1e-8:
        return qi * np.exp(-d * t)
    return qi / ((1 + b * d * t)**(1.0 / b))

def fit_arps(times, rates, initial_guess=(500, 0.05, 0.3)):
    # times in months (0,1,2,...), rates in same units as qi
    def loss(params):
        qi, d, b = params
        pred = arps_rate(qi, d, b, times)
        return np.sqrt(np.mean((rates - pred)**2))
    bounds = [(1e-6, max(rates)*10), (1e-6, 1.0), (0.0, 1.5)]
    res = minimize(loss, x0=initial_guess, bounds=bounds, method='L-BFGS-B')
    if res.success:
        return dict(qi=res.x[0], d=res.x[1], b=res.x[2], rmse=res.fun)
    else:
        return None

def compute_eur_from_params(qi, d, b, months=6000):
    # integrate rate over months to get total volume (monthly units)
    t = np.arange(0, months)
    rates = arps_rate(qi, d, b, t)
    return rates.sum()

summary_rows = []
for wid, grp in prod.groupby('Well_ID'):
    g = grp.sort_values('Date')
    g['MonthIdx'] = np.arange(len(g))
    times = g['MonthIdx'].values
    rates = g['Oil_bopd'].values
    if len(rates) < 12 or np.all(rates<=0):
        continue
    fit = fit_arps(times, rates, initial_guess=(max(rates), 0.03, 0.3))
    if fit is None:
        continue
    eur = compute_eur_from_params(fit['qi'], fit['d'], fit['b'], months=1200)
    summary_rows.append({'Well_ID': wid, 'qi': fit['qi'], 'd': fit['d'], 'b': fit['b'], 'rmse': fit['rmse'], 'EUR_bopd_months': eur})

summary_df = pd.DataFrame(summary_rows)
summary_df.to_csv(OUT_DIR / 'dca_summary.csv', index=False)
print('DCA summary rows:', summary_df.shape[0])
summary_df.head()


## 11. Save model artifacts and summary
Models saved to `outputs/` directory.

In [ ]:
joblib.dump(rf_por, OUT_DIR / 'rf_porosity.joblib')
joblib.dump(rf_perm, OUT_DIR / 'rf_permeability.joblib')
if 'clf' in globals():
    joblib.dump(clf, OUT_DIR / 'rf_facies.joblib')
print('Saved models to', OUT_DIR)
summary_df.head()


## 12. Example: Predict porosity & permeability for all log samples (batch inference)


In [ ]:
# Prepare features from logs for batch prediction
logs_feat = logs.copy()
logs_feat['GR_over_RHOB'] = logs_feat['GR'] / (logs_feat['RHOB'] + 1e-6)
logs_feat['NPHI_times_GR'] = logs_feat['NPHI'] * logs_feat['GR']
feat_cols = ['GR','NPHI','RHOB','DT','RES','GR_over_RHOB','NPHI_times_GR']
X_logs = logs_feat[feat_cols].values
X_logs_s = scaler.transform(X_logs)
logs_feat['Pred_Porosity'] = rf_por.predict(X_logs_s)
logs_feat['Pred_logPerm'] = rf_perm.predict(X_logs_s)
logs_feat['Pred_Perm_mD'] = np.expm1(logs_feat['Pred_logPerm'])
logs_feat[['Well_ID','Depth_m','Pred_Porosity','Pred_Perm_mD']].head()
logs_feat.to_csv(OUT_DIR / 'logs_with_predictions.csv', index=False)
print('Wrote logs_with_predictions.csv')


## 13. Next steps and tips
- Tune hyperparameters with GridSearchCV for better performance.
- Add spatial interpolation (Kriging) for field maps.
- Add Monte Carlo UQ for DCA parameters to estimate EUR distributions.
- Replace synthetic data with actual field data and re-run the pipeline.
